## Packages

In [78]:
import pickle
import pandas as pd

## Classes and Functions

## Dependencies

In [79]:
X_train = pd.read_csv('../../assets/data/splits/train/preprocessed.csv')
X_val = pd.read_csv('../../assets/data/splits/val/preprocessed.csv')

In [80]:
y_train = pd.read_csv('../../assets/data/splits/train/raw.csv')['label']
y_val = pd.read_csv('../../assets/data/splits/val/raw.csv')['label']

In [81]:
with open('../../assets/traditional_assets/cv_vec.pkl', 'rb') as fout:
    cv_vec = pickle.load(fout)


In [82]:
with open('../../assets/traditional_assets/tfidf_vec.pkl', 'rb') as fout:
    tfidf_vec = pickle.load(fout)

## Grid Params

### KNN

In [83]:
from sklearn.neighbors import KNeighborsClassifier

knn_params = {
    'n_neighbors': [11, 21, 40, 60, 80, 100],
    'metric': ['cosine'],
    # 'weights': ['distance']
    'weights': ['uniform', 'distance'],
    # 'metric': ['cosine']
}

### SVM

In [84]:
from sklearn.svm import SVC

svm_params = {
    'C': [10, 50, 100],
    'kernel': ['rbf', 'linear', 'poly'],
}

### Naive Bayes

In [85]:
from sklearn.naive_bayes import MultinomialNB

nb_params = {
    'alpha': [0.1, 1, 10],
    'fit_prior': [True, False]
}

### Logistic Regression

In [86]:
from sklearn.linear_model import LogisticRegression

lr_params = {
    'penalty': ['l2', None],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'newtgon-cg', 'sag', 'saga']
}

## Tuning and Evaluation

In [87]:
import numpy as np
from sklearn.model_selection import PredefinedSplit

split_index = [-1] * len(X_train) + [0] * len(X_val)

X = pd.DataFrame(X_train)
X = X.append(X_val)

y = np.concatenate((y_train, y_val), axis=0)
pds = PredefinedSplit(test_fold=split_index)





C:\Users\Guilherme\AppData\Local\Temp\ipykernel_56124\1690835414.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(X_val)


### Count Vectorizer

In [88]:
X_cv = cv_vec.transform(X['title']).toarray()
pd.DataFrame(X_cv, columns=cv_vec.get_feature_names_out())

MemoryError: Unable to allocate 1.25 GiB for an array with shape (16210, 10390) and data type int64

In [ ]:
from sklearn.model_selection import GridSearchCV

# from hypopt import GridSearch

model_params = ([KNeighborsClassifier(), SVC(), MultinomialNB(), LogisticRegression()],
                [knn_params, svm_params, nb_params, lr_params])

list_best_models_params = []
for model, params in zip(model_params[0], model_params[1]):
    gs = GridSearchCV(model,
                      param_grid=params,
                      )

    gs.fit(X_cv, y)
    print(f"Best CV results for {model.__class__.__name__}")
    print("Best Score of train set: " + str(gs.best_score_))
    print("Best estimator: " + str(gs.best_estimator_))
    print("Best parameter set: " + str(gs.best_params_))

    store_best_model_configs = {
        'model_name': model.__class__.__name__,
        'best_score': gs.best_score_,
        'best_estimator': gs.best_estimator_,
        'best_params': gs.best_params_
    }

    list_best_models_params.append(store_best_model_configs)

df_best_models_params = pd.DataFrame(list_best_models_params)
df_best_models_params.to_csv('../../assets/traditional_assets/best_models_params_cv.csv', index=False)

df_best_models_params
# cv_best_model = gs.best_estimator_
# print("Test Score: " + str(gs.score(X_val_cv, y_val)))
# print("----------------------------------------------------")

In [ ]:

cv_best_model = gs.best_estimator_
cv_best_model

### TF-IDF

In [ ]:
# X_train_tfidf = tfidf_vec.transform(X_train['title'])
# X_val_tfidf = tfidf_vec.transform(X_val['title'])
# X_train_tfidf
X_tfidf = tfidf_vec.transform(X['title']).toarray()
pd.DataFrame(X_cv, columns=tfidf_vec.get_feature_names_out())

In [ ]:
# from sklearn.model_selection import GridSearchCVCV

model_params = ([KNeighborsClassifier(), SVC(), MultinomialNB(), LogisticRegression()],
                [knn_params, svm_params, nb_params, lr_params])

list_best_models_params = []
for model, params in zip(model_params[0], model_params[1]):
    gs = GridSearchCV(model,
                      param_grid=params,
                      )
    gs.fit(X_tfidf, y)
    print(f"Best TF-IDF results for {model.__class__.__name__}")
    print("Best Score on train set: " + str(gs.best_score_))
    print("Best estimator: " + str(gs.best_estimator_))
    print("Best parameter set: " + str(gs.best_params_) + "\n")
    store_best_model_configs = {
        'model_name': model.__class__.__name__,
        'best_score': gs.best_score_,
        'best_estimator': gs.best_estimator_,
        'best_params': gs.best_params_
    }

    list_best_models_params.append(store_best_model_configs)

df_best_models_params = pd.DataFrame(list_best_models_params)
df_best_models_params.to_csv('../../assets/traditional_assets/best_models_params_tfidf.csv', index=False)
df_best_models_params


# decide_best_model =
# print("Test Score: " + str(gs.score(X_val, y_val)))
# print("----------------------------------------------------")


In [ ]:
tfidf_best_model = gs.best_estimator_
tfidf_best_model

## Outputs

In [ ]:
with open('../../assets/traditional_assets/cv_set,pkl', 'wb') as fout:
    pickle.dump((cv_vec, cv_best_model), fout)

with open('../../assets/traditional_assets/tfidf_set.pkl', 'wb') as fout:
    pickle.dump((tfidf_vec, cv_best_model), fout)
